In [22]:
import pandas as pd
import numpy as np
import re
import os

#### **Common Utility Functions**

In [25]:
def remove_rows_with_empty_columns(df):
    # Select all columns except the first one
    other_columns = df.iloc[:, 1:]
    
    # Create a boolean mask for rows where all columns except the first one are NaN
    rows_to_drop = other_columns.isna().all(axis=1)
    
    # Drop those rows from the DataFrame
    df_cleaned = df[~rows_to_drop]
    
    return df_cleaned

## (1) Convert Excel to CSVs ('zillow_data_dictionary.xlsx')

In [21]:
file_path = './data/zillow_data_dictionary.xlsx'

def format_column_name(column_name):
    column_name = re.sub(r'ID$', '', column_name)
    formatted_name = re.sub(r'(?<!^)(?=[A-Z])', '_', column_name).lower()
    formatted_name += '_df'
    return formatted_name

excel_data = pd.read_excel(file_path, sheet_name=None)

for sheet_name, data in excel_data.items():
    sheet_name = format_column_name(sheet_name)
    csv_file_name = f"./inputs/{sheet_name}.csv"
    data.to_csv(csv_file_name, index=False)
    print(f"Saved {sheet_name} as {csv_file_name}")

os.path.exists("./inputs/data_dictionary_df.csv") and os.remove("./inputs/data_dictionary_df.csv")

Saved data _dictionary_df as ./inputs/data _dictionary_df.csv
Saved heating_or_system_type_df as ./inputs/heating_or_system_type_df.csv
Saved property_land_use_type_df as ./inputs/property_land_use_type_df.csv
Saved story_type_df as ./inputs/story_type_df.csv
Saved air_conditioning_type_df as ./inputs/air_conditioning_type_df.csv
Saved architectural_style_type_df as ./inputs/architectural_style_type_df.csv
Saved type_construction_type_df as ./inputs/type_construction_type_df.csv
Saved building_class_type_df as ./inputs/building_class_type_df.csv


## (2) Split Master CSV to Tables ('properties_2017.csv')

In [5]:
df = pd.read_csv('./data/properties_2017.csv')

C:\Users\Acer\AppData\Local\Temp\ipykernel_16416\4187841073.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/properties_2017.csv')


In [6]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [7]:
df.shape

(2985217, 58)

#### **1. Property Table**

In [8]:
property_columns = [
    'parcelid', 'latitude', 'longitude', 'yearbuilt', 'regionidcounty',
    'regionidcity', 'regionidzip', 'regionidneighborhood'
]

# 1. Filter Required Columns
property_df = df[property_columns]
property_df.head()

,parcelid,latitude,longitude,lotsizesquarefeet,numberofstories,yearbuilt,regionidcounty,regionidcity,regionidzip,regionidneighborhood
0,10754147,34144442.0,-118654084.0,85768.0,NaN,NaN,3101.0,37688.0,96337.0,NaN
1,10759547,34140430.0,-118625364.0,4083.0,NaN,NaN,3101.0,37688.0,96337.0,NaN
2,10843547,33989359.0,-118394633.0,63085.0,1.0,1959.0,3101.0,51617.0,96095.0,NaN
3,10859147,34148863.0,-118437206.0,7521.0,1.0,1948.0,3101.0,12447.0,96424.0,27080.0
4,10879947,34194168.0,-118385816.0,8512.0,1.0,1947.0,3101.0,12447.0,96450.0,46795.0


In [26]:
# 2. Remove All Column Missing
property_df = remove_rows_with_empty_columns(property_df)

In [28]:
# 3. Drop All Column Duplicates
property_df = property_df.drop_duplicates()

In [30]:
# 4. Check for Missing Values
property_df.isna().sum()

parcelid                      0
latitude                      0
longitude                     0
lotsizesquarefeet        269774
numberofstories         2296609
yearbuilt                 44901
regionidcounty                0
regionidcity              59196
regionidzip                9782
regionidneighborhood    1825544
dtype: int64

In [32]:
# 5. Check Whether 'Many' Dependancy Exists
property_df['parcelid'].value_counts().unique()

array([1])

Since this is our Primary Table, there are no relationships exist.

In [14]:
# 6. Save DataFrame
property_df.to_csv('./inputs/property_table.csv', index=False)

#### **2. Building Table**

In [33]:
building_columns = [
    'parcelid', 'bedroomcnt', 'bathroomcnt', 'calculatedbathnbr', 
    'threequarterbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 
    'finishedsquarefeet6', 'finishedsquarefeet12', 'finishedsquarefeet13', 
    'finishedsquarefeet15', 'finishedsquarefeet50', 'buildingqualitytypeid',
    'buildingclasstypeid', 'storytypeid', 'typeconstructiontypeid'
]

# 1. Filter Columns
building_df = df[building_columns]

# 2. Remove All Column Missing
building_df = remove_rows_with_empty_columns(building_df)

# 3. Drop All Column Duplicates
building_df = building_df.drop_duplicates()


In [34]:
building_df.shape

(2982278, 16)

In [35]:
# 4. Check for Missing Values
building_df.isna().sum()

parcelid                              0
bedroomcnt                            6
bathroomcnt                          18
calculatedbathnbr                114217
threequarterbathnbr             2665921
finishedfloor1squarefeet        2778520
calculatedfinishedsquarefeet      42158
finishedsquarefeet6             2960796
finishedsquarefeet12             261492
finishedsquarefeet13            2974607
finishedsquarefeet15            2792093
finishedsquarefeet50            2778520
buildingqualitytypeid           1040883
buildingclasstypeid             2969547
storytypeid                     2980655
typeconstructiontypeid          2975532
dtype: int64

In [41]:
building_df.notna().sum()

parcelid                        2982278
bedroomcnt                      2982272
bathroomcnt                     2982260
calculatedbathnbr               2868061
threequarterbathnbr              316357
finishedfloor1squarefeet         203758
calculatedfinishedsquarefeet    2940120
finishedsquarefeet6               21482
finishedsquarefeet12            2720786
finishedsquarefeet13               7671
finishedsquarefeet15             190185
finishedsquarefeet50             203758
buildingqualitytypeid           1941395
buildingclasstypeid               12731
storytypeid                        1623
typeconstructiontypeid             6746
dtype: int64

In [36]:
# 5. Check Whether 'Many' Dependancy Exists
property_df['parcelid'].value_counts().unique()

array([1])

#### **2. Amenities Table**

In [ ]:
amenities_columns = [
    'parcelid', 'fireplacecnt', 'fireplaceflag', 'fullbathcnt', 
    'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'poolcnt', 
    'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
    'decktypeid', 'yardbuildingsqft17', 'yardbuildingsqft26'
]

amenities_df = df[amenities_columns]

#### **3. Tax Table** 

In [ ]:
tax_columns = [
    'parcelid', 'taxvaluedollarcnt', 'structuretaxvaluedollarcnt', 
    'landtaxvaluedollarcnt', 'taxamount', 'assessmentyear', 
    'taxdelinquencyflag', 'taxdelinquencyyear'
]

tax_df = df[tax_columns]

#### **4. Zoning Table**  

In [ ]:
zoning_columns = [
    'parcelid', 'propertycountylandusecode', 'propertylandusetypeid', 
    'propertyzoningdesc', 'rawcensustractandblock', 'censustractandblock'
]

zoning_df = df[zoning_columns]